In [1]:
import numpy as np
import matplotlib.pyplot as plt

from mvdr.toy_data.joint_fact_model import sample_joint_factor_model
from mvdr.linalg_utils import rand_orthog
from mvdr.principal_angles import get_principal_angles, subspace_dist
from mvdr.mcca.mcca import MCCA
from mvdr.mcca.k_mcca import KMCCA

# sample data from a joint factor model 

In [2]:
joint_rank = 3
n_samples = 1000
n_features=[10, 20, 30]
n_blocks = len(n_features)

Xs, U_true, Ws_true = sample_joint_factor_model(n_samples=n_samples, n_features=n_features,
                                                joint_rank=joint_rank,
                                                m=5, noise_std=1, # these control the difficulty of the problem
                                                random_state=23)

# Fit MCCA

In [3]:
# the default is no regularization meaning this is SUMCORR-AVGVAR MCCA
mcca = MCCA(n_components=joint_rank)

In [4]:
# the fit-transform method outputs the common normalized scores (CNS)
common_normalized_scores = mcca.fit_transform(Xs)

# applying transform to the original data also gives the CNS
common_normalized_scores = mcca.transform(Xs)

# the block information (e.g. block loadings) are stored in the blocks_ attribute
b = 0
mcca.blocks_[b].block_loadings_
mcca.blocks_[b].block_scores_

# the blocks_ attribute can project new data from each view
np.allclose(mcca.blocks_[b].transform(Xs[b]),
            mcca.blocks_[b].block_scores_)

True

In [5]:
# lets see how accurately we can estimate the true loadings!
def summarize_loading_acc(mcca, Ws_true):
    """
    Prints the vector of principal angles comparing the
    subspace spanned by the estimated block loadings
    with the subspace spanned by the true block loadings.
    """
    
    for b in range(mcca.n_blocks_):
        est_block_loadings = mcca.block_loadings_[b]
        true_block_loadings = Ws_true[b]
        theta = get_principal_angles(est_block_loadings, true_block_loadings,
                                     is_ortho=False, deg=True)
        print("Principal angles for view {} are {} degrees".format(b, theta))
        
summarize_loading_acc(mcca, Ws_true)

Principal angles for view 0 are [3.25948008 5.48505734 7.66368219] degrees
Principal angles for view 1 are [ 7.05754975 10.33635925 11.51632033] degrees
Principal angles for view 2 are [10.3777501  12.81570617 13.96373189] degrees


# MCCA with regularization

We can add regularization with the `regs` argument to handle high-dimensional data.

In [6]:
# regularization value of .1 for each block
mcca = MCCA(n_components=joint_rank, regs=.5).fit(Xs)

# we can provide different regularization values for each block 
# by passing in a list
# mcca = MCCA(n_components=joint_rank, regs=[.1, .2, .3]).fit(Xs)

# a simple default regularization valuae can be obtained
# using the Ledoit Wolf method for regularized covariance matrix estimation
# mcca = MCCA(n_components=joint_rank, regs='lw').fit(Xs)

summarize_loading_acc(mcca, Ws_true)

Principal angles for view 0 are [3.25667814 5.47941317 7.65637561] degrees
Principal angles for view 1 are [ 7.05073776 10.32607509 11.50648371] degrees
Principal angles for view 2 are [10.3677296  12.80431272 13.95209547] degrees


# Informative MCCA: PCA then MCCA

We can also handle high-dimensional data with i-MCCA. We first compute a low rank PCA for each block, then run MCCA on the reduced data.

In [7]:
# i-MCCA where we first extract the first 5 PCs from each data block
mcca = MCCA(n_components=joint_rank, signal_ranks=[5, 5, 5]).fit(Xs)
summarize_loading_acc(mcca, Ws_true)

Principal angles for view 0 are [1.08226889 1.52126199 5.58547933] degrees
Principal angles for view 1 are [2.11139036 3.15782628 4.37740522] degrees
Principal angles for view 2 are [2.56143066 4.01880307 5.75075158] degrees


#  Kernel-MCCA

We can compute kernel MCCA with the KMCCA() object.

In [8]:
# fit kernel MCCA with a linear kernel
kmcca = KMCCA(n_components=joint_rank, kernel='linear').fit(Xs)

In [9]:
# the common normalized scores or kmcca with linear kernel
# should be the same as the common normalized scores for MCCA
mcca = MCCA(n_components=joint_rank).fit(Xs)
np.allclose(mcca.common_norm_scores_, kmcca.common_norm_scores_)

True